In [1]:
from ipsl_dataset import IPSL_DCPP
import torch
train = IPSL_DCPP('train',generate_statistics=True)
train_dataloader = torch.utils.data.DataLoader(train,batch_size=1,shuffle=True,num_workers=1)
batch = next(iter(train_dataloader))

150it [00:24,  6.24it/s]


In [2]:
from pangu import PanguWeather
model = PanguWeather()

/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu4/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu4/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1705513624012/work/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
x = [0,1,2,3,4,5,6,7,8]
x_sub = x[:-3]
x[len(x_sub)+2]

8

In [3]:
model.forward(batch)

surface dims torch.Size([1, 135, 143, 144])
plev dims torch.Size([1, 8, 19, 143, 144])
embedded surface shape torch.Size([1, 192, 36, 36])
embedded upper_air shape torch.Size([1, 192, 10, 36, 36])
after concat torch.Size([1, 192, 11, 36, 36])
right before layer1 torch.Size([1, 14256, 192])
in earth specific block
input resol (11, 36, 36)
x shape torch.Size([1, 14256, 192])
RAM Used (GB): 27.163389952
in earth specific block
input resol (11, 36, 36)
x shape torch.Size([1, 14256, 192])
RAM Used (GB): 28.330520576
downsampled torch.Size([1, 3564, 384])
in earth specific block
input resol (11, 18, 18)
x shape torch.Size([1, 3564, 384])
RAM Used (GB): 29.7996288
in earth specific block
input resol (11, 18, 18)
x shape torch.Size([1, 3564, 384])
RAM Used (GB): 29.433696256
in earth specific block
input resol (11, 18, 18)
x shape torch.Size([1, 3564, 384])
RAM Used (GB): 28.764205056
in earth specific block
input resol (11, 18, 18)
x shape torch.Size([1, 3564, 384])
RAM Used (GB): 29.25271859

{'latent': tensor([[[nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          ...,
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan],
          [nan, nan, nan,  ..., nan, nan, nan]]], grad_fn=<AddBackward0>),
 'next_state_level': tensor([[[[[nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            ...,
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan]],
 
           [[nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            ...,
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan],
            [nan, nan, nan,  ..., nan, nan, nan]],
 
